# Lists and Tuples

*📚 Computer Programming II · 👨‍🏫 Dr. Arif Solmaz*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_05.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Understand lists as ordered, mutable collections
- Create lists using various methods
- Access elements using positive and negative indexing
- Extract portions using slicing with start:stop:step
- Use list methods: append, insert, remove, pop, sort
- Apply built-in functions: len, sum, min, max, sorted
- Write efficient list comprehensions
- Work with nested lists (2D arrays/matrices)
- Understand tuples as immutable sequences
- Apply tuple unpacking and packing
- Choose between lists and tuples appropriately
- Apply data structures to engineering problems

---
## Part 1: Introduction to Lists


A **list** is one of Python's most versatile and commonly used data structures. It's an **ordered**, **mutable** collection that can hold items of any data type. Lists are fundamental to programming and are used extensively in engineering for storing sensor data, measurements, coordinates, and more.

### What is a List?

Think of a list as a container that holds multiple items in a specific order. Each item has a position (index), and you can add, remove, or change items at any time.

| Feature | Description | Example |
| --- | --- | --- |
| **Ordered** | Items maintain their position/sequence | [1, 2, 3] stays in that order |
| **Mutable** | Can be modified after creation | Can add/remove/change items |
| **Heterogeneous** | Can contain different data types | [1, "hello", 3.14, True] |
| **Dynamic** | Can grow or shrink in size | No fixed size limit |
| **Indexed** | Each item has a position number | First item is at index 0 |

### Creating Lists

There are several ways to create lists in Python:

**Figure 1.1: Creating Lists**

In [ ]:
# Method 1: Square bracket notation (most common)
empty_list = []
numbers = [1, 2, 3, 4, 5]
fruits = ["apple", "banana", "cherry"]
mixed = [1, "hello", 3.14, True, None]

print("Empty list:", empty_list)
print("Numbers:", numbers)
print("Fruits:", fruits)
print("Mixed types:", mixed)
print("\nType:", type(numbers))

**Figure 1.2: Other Ways to Create Lists**

In [ ]:
# Method 2: list() constructor
from_string = list("Python")
print("From string:", from_string)

from_range = list(range(1, 6))
print("From range:", from_range)

from_tuple = list((10, 20, 30))
print("From tuple:", from_tuple)

# Method 3: List multiplication
zeros = [0] * 5
print("\nZeros:", zeros)

pattern = [1, 2] * 4
print("Pattern:", pattern)

### List Visualization

Understanding how lists are stored in memory helps you work with them effectively:

```

fruits = ["apple", "banana", "cherry", "date"]

Index:      0         1          2         3
         +--------+--------+--------+--------+
Value:   | apple  | banana | cherry |  date  |
         +--------+--------+--------+--------+
Neg Idx:   -4        -3        -2        -1
```

> 💡 **Note:** **Memory Tip:** Lists store references to objects, not the objects themselves. This means lists can efficiently hold large objects without copying them.

---
## Part 2: Indexing and Access


Every element in a list has a unique position called an **index**. Python uses zero-based indexing, meaning the first element is at index 0.

### Positive Indexing

**Figure 2.1: Positive Indexing**

In [ ]:
# Access elements by positive index
fruits = ["apple", "banana", "cherry", "date", "elderberry"]

print("Index 0:", fruits[0])    # First element
print("Index 1:", fruits[1])    # Second element
print("Index 2:", fruits[2])    # Third element
print("Index 4:", fruits[4])    # Fifth (last) element

# Get the length
print(f"\nList length: {len(fruits)}")
print(f"Last valid index: {len(fruits) - 1}")

### Negative Indexing

Python also supports negative indexing, which counts from the end of the list:

**Figure 2.2: Negative Indexing**

In [ ]:
# Access elements by negative index
fruits = ["apple", "banana", "cherry", "date", "elderberry"]

print("Index -1:", fruits[-1])   # Last element
print("Index -2:", fruits[-2])   # Second to last
print("Index -3:", fruits[-3])   # Third to last
print("Index -5:", fruits[-5])   # First element

# Useful for getting the last element
print(f"\nLast fruit: {fruits[-1]}")

### Modifying Elements

**Figure 2.3: Modifying List Elements**

In [ ]:
# Lists are mutable - we can change elements
colors = ["red", "green", "blue"]
print("Original:", colors)

# Change an element
colors[1] = "yellow"
print("After colors[1] = 'yellow':", colors)

# Change last element
colors[-1] = "purple"
print("After colors[-1] = 'purple':", colors)

# Change first element
colors[0] = "orange"
print("After colors[0] = 'orange':", colors)

### Checking Membership

**Figure 2.4: Membership Testing**

In [ ]:
# Check if an element exists in a list
fruits = ["apple", "banana", "cherry"]

print("'banana' in list:", "banana" in fruits)
print("'grape' in list:", "grape" in fruits)
print("'grape' NOT in list:", "grape" not in fruits)

# Finding index
print(f"\nIndex of 'cherry': {fruits.index('cherry')}")

# Counting occurrences
grades = [85, 90, 85, 78, 92, 85]
print(f"85 appears {grades.count(85)} times")

> 💡 **Note:** Accessing an index that doesn't exist raises an `IndexError`. Always check the list length or use try/except when working with dynamic indices.

---
## Part 3: Slicing Techniques


**Slicing** allows you to extract a portion (subset) of a list. It's one of Python's most powerful features for working with sequences.

### Slice Syntax

```
list[start:stop:step]

• start: Starting index (inclusive, default 0)
• stop: Ending index (exclusive, default len(list))
• step: Step size (default 1)
```

#### Basic Slicing

**Figure 3.1: Basic Slicing**

In [ ]:
# Basic slicing examples
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print("Full list:", numbers)

print("\n[2:5] =", numbers[2:5])     # Index 2, 3, 4
print("[:4] =", numbers[:4])         # First 4 elements
print("[6:] =", numbers[6:])         # From index 6 to end
print("[:] =", numbers[:])           # Full copy
print("[-3:] =", numbers[-3:])       # Last 3 elements

#### Slicing with Step

**Figure 3.2: Slicing with Step**

In [ ]:
# Using step in slicing
numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

print("Every 2nd [::2] =", numbers[::2])
print("Every 3rd [::3] =", numbers[::3])
print("[1:8:2] =", numbers[1:8:2])

# Reverse with negative step
print("\nReversed [::-1] =", numbers[::-1])
print("Every 2nd reversed [::-2] =", numbers[::-2])

#### Slice Assignment

**Figure 3.3: Slice Assignment**

In [ ]:
# You can assign to slices!
letters = ['a', 'b', 'c', 'd', 'e']
print("Original:", letters)

# Replace a slice
letters[1:3] = ['X', 'Y', 'Z']
print("After [1:3] = ['X','Y','Z']:", letters)

# Delete with empty list
letters[2:4] = []
print("After [2:4] = []:", letters)

# Insert with slice
numbers = [1, 2, 5, 6]
numbers[2:2] = [3, 4]  # Insert at position 2
print("\nInsert [3,4] at position 2:", numbers)

#### Engineering: Time-Series Data

**Figure 3.4: Time-Series Slicing**

In [ ]:
# Extract time periods from 24-hour temperature data
hourly_temps = [18, 17, 16, 16, 15, 15, 16, 18, 20, 22, 24, 26,
                28, 29, 30, 29, 28, 26, 24, 22, 21, 20, 19, 18]

morning = hourly_temps[6:12]      # 6AM - 11AM
afternoon = hourly_temps[12:18]   # 12PM - 5PM
evening = hourly_temps[18:24]     # 6PM - 11PM

print(f"Morning temps (06:00-11:00): {morning}")
print(f"Afternoon temps (12:00-17:00): {afternoon}")
print(f"Evening temps (18:00-23:00): {evening}")

print(f"\nMorning avg: {sum(morning)/len(morning):.1f}°C")
print(f"Afternoon avg: {sum(afternoon)/len(afternoon):.1f}°C")
print(f"Peak temp: {max(hourly_temps)}°C at {hourly_temps.index(max(hourly_temps))}:00")

---
## Part 4: List Methods


Lists come with many built-in methods for modification. These methods change the list in place (mutate it).

| Method | Description | Example |
| --- | --- | --- |
| `append(x)` | Add item to end | `list.append(4)` |
| `insert(i, x)` | Insert at position | `list.insert(0, "first")` |
| `extend(iter)` | Add all from iterable | `list.extend([4, 5])` |
| `remove(x)` | Remove first occurrence | `list.remove("apple")` |
| `pop(i)` | Remove and return at index | `list.pop(0)` |
| `clear()` | Remove all items | `list.clear()` |
| `sort()` | Sort in place | `list.sort()` |
| `reverse()` | Reverse in place | `list.reverse()` |
| `copy()` | Return shallow copy | `new = list.copy()` |

### Adding Elements

**Figure 4.1: Adding Elements**

In [ ]:
# append() - add one item to end
fruits = ["apple", "banana"]
print("Original:", fruits)

fruits.append("cherry")
print("After append('cherry'):", fruits)

# insert() - add at specific position
fruits.insert(0, "apricot")
print("After insert(0, 'apricot'):", fruits)

fruits.insert(2, "blueberry")
print("After insert(2, 'blueberry'):", fruits)

# extend() - add multiple items
fruits.extend(["date", "elderberry"])
print("After extend(['date', 'elderberry']):", fruits)

### Removing Elements

**Figure 4.2: Removing Elements**

In [ ]:
# Various ways to remove elements
numbers = [10, 20, 30, 40, 50, 30]
print("Original:", numbers)

# remove() - remove first occurrence of value
numbers.remove(30)
print("After remove(30):", numbers)

# pop() - remove and return by index
last = numbers.pop()       # Remove last
print(f"Popped last: {last}, list: {numbers}")

first = numbers.pop(0)     # Remove first
print(f"Popped first: {first}, list: {numbers}")

# del - delete by index or slice
del numbers[1]
print("After del numbers[1]:", numbers)

# clear() - remove all
numbers.clear()
print("After clear():", numbers)

### Sorting

**Figure 4.3: Sorting Lists**

In [ ]:
# sort() - modifies list in place
numbers = [64, 34, 25, 12, 22, 11, 90]
print("Original:", numbers)

numbers.sort()
print("Sorted ascending:", numbers)

numbers.sort(reverse=True)
print("Sorted descending:", numbers)

# sorted() - returns new list, original unchanged
words = ["banana", "Apple", "cherry", "Date"]
print("\nOriginal words:", words)
print("sorted():", sorted(words))
print("sorted(key=str.lower):", sorted(words, key=str.lower))
print("Original still:", words)

### Reversing

**Figure 4.4: Reversing Lists**

In [ ]:
# reverse() - modifies list in place
letters = ['a', 'b', 'c', 'd', 'e']
print("Original:", letters)

letters.reverse()
print("After reverse():", letters)

# reversed() - returns iterator
numbers = [1, 2, 3, 4, 5]
print("\nOriginal numbers:", numbers)
print("list(reversed()):", list(reversed(numbers)))
print("Slice [::-1]:", numbers[::-1])
print("Original still:", numbers)

> 💡 **Note:** **sort() vs sorted():** `sort()` modifies the original list and returns None. `sorted()` returns a new sorted list and leaves the original unchanged.

---
## Part 5: List Operations and Built-in Functions


### List Concatenation and Repetition

**Figure 5.1: Concatenation and Repetition**

In [ ]:
# Concatenation with +
list1 = [1, 2, 3]
list2 = [4, 5, 6]
combined = list1 + list2
print("list1 + list2 =", combined)

# Repetition with *
zeros = [0] * 5
print("[0] * 5 =", zeros)

pattern = [1, 2] * 3
print("[1, 2] * 3 =", pattern)

# += operator (in-place)
numbers = [1, 2]
numbers += [3, 4]
print("\n[1,2] += [3,4] =", numbers)

### Built-in Functions for Lists

**Figure 5.2: Built-in Functions**

In [ ]:
# Common built-in functions
numbers = [45, 22, 89, 34, 67, 12, 78]

print("List:", numbers)
print(f"len():  {len(numbers)}")
print(f"min():  {min(numbers)}")
print(f"max():  {max(numbers)}")
print(f"sum():  {sum(numbers)}")
print(f"Average: {sum(numbers)/len(numbers):.2f}")

# any() and all()
bools = [True, False, True]
print(f"\nany({bools}): {any(bools)}")
print(f"all({bools}): {all(bools)}")

# Check if any number > 50
print(f"any(n > 50): {any(n > 50 for n in numbers)}")
print(f"all(n > 0): {all(n > 0 for n in numbers)}")

### Iterating with enumerate() and zip()

**Figure 5.3: enumerate and zip**

In [ ]:
# enumerate() - get index and value
fruits = ["apple", "banana", "cherry"]
print("Using enumerate():")
for index, fruit in enumerate(fruits):
    print(f"  {index}: {fruit}")

# Start from different number
print("\nStarting from 1:")
for num, fruit in enumerate(fruits, start=1):
    print(f"  {num}. {fruit}")

# zip() - combine multiple lists
names = ["Alice", "Bob", "Charlie"]
scores = [85, 92, 78]
grades = ["B", "A", "C"]

print("\nUsing zip():")
for name, score, grade in zip(names, scores, grades):
    print(f"  {name}: {score} ({grade})")

### List Copying

**Figure 5.4: Shallow vs Deep Copy**

In [ ]:
# Be careful with assignment!
original = [1, 2, 3]
reference = original  # NOT a copy! Same object

reference[0] = 999
print("After changing reference[0]:")
print("  original:", original)
print("  reference:", reference)

# Proper ways to copy
original = [1, 2, 3]
copy1 = original.copy()
copy2 = original[:]
copy3 = list(original)

copy1[0] = 100
print("\nAfter changing copy1[0]:")
print("  original:", original)
print("  copy1:", copy1)

---
## Part 6: List Comprehensions


A **list comprehension** is a concise way to create lists. It's one of Python's most powerful and elegant features.

### Basic Syntax

```
[expression for item in iterable]

# With condition:
[expression for item in iterable if condition]
```

#### Basic Comprehensions

**Figure 6.1: Basic Comprehensions**

In [ ]:
# Traditional loop vs comprehension
# Loop version:
squares_loop = []
for x in range(1, 6):
    squares_loop.append(x ** 2)
print("Loop:", squares_loop)

# Comprehension version:
squares_comp = [x ** 2 for x in range(1, 6)]
print("Comprehension:", squares_comp)

# More examples
doubles = [n * 2 for n in range(1, 6)]
print("Doubles:", doubles)

cubes = [n ** 3 for n in range(1, 5)]
print("Cubes:", cubes)

#### Comprehensions with Conditions

**Figure 6.2: Filtering with Conditions**

In [ ]:
# Filter with if condition
numbers = range(1, 11)

evens = [n for n in numbers if n % 2 == 0]
print("Even numbers:", evens)

odds = [n for n in numbers if n % 2 != 0]
print("Odd numbers:", odds)

# Multiple conditions
divisible_by_2_or_3 = [n for n in range(1, 21) if n % 2 == 0 or n % 3 == 0]
print("Divisible by 2 or 3:", divisible_by_2_or_3)

# Filter strings
words = ["apple", "banana", "cherry", "date", "elderberry"]
long_words = [w for w in words if len(w) > 5]
print("Words > 5 chars:", long_words)

#### Conditional Expressions

**Figure 6.3: If-Else in Comprehensions**

In [ ]:
# if-else in the expression (not filter)
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Label each number
labels = ["even" if n % 2 == 0 else "odd" for n in numbers]
print("Labels:", labels)

# Replace negatives with zero
data = [-5, 3, -2, 8, -1, 6]
positive = [x if x > 0 else 0 for x in data]
print("Positive only:", positive)

# Categorize grades
scores = [45, 72, 88, 55, 91, 67]
grades = ["Pass" if s >= 60 else "Fail" for s in scores]
print("Scores:", scores)
print("Grades:", grades)

#### Engineering: Data Processing

**Figure 6.4: Engineering Applications**

In [ ]:
# Sensor data processing
raw_readings = [23.5, 24.1, -999, 23.8, 24.0, -999, 23.9]
# -999 indicates sensor error

# Filter out errors
valid = [r for r in raw_readings if r != -999]
print("Valid readings:", valid)
print(f"Average: {sum(valid)/len(valid):.2f}")

# Convert Celsius to Fahrenheit
celsius = [0, 10, 20, 30, 40]
fahrenheit = [c * 9/5 + 32 for c in celsius]
print(f"\nCelsius: {celsius}")
print(f"Fahrenheit: {fahrenheit}")

# Normalize data (0-1 scale)
data = [25, 50, 75, 100, 125]
min_val, max_val = min(data), max(data)
normalized = [(x - min_val) / (max_val - min_val) for x in data]
print(f"\nOriginal: {data}")
print(f"Normalized: {normalized}")

---
## Part 7: Nested Lists (2D Arrays)


A **nested list** is a list containing other lists. This creates a 2D structure similar to matrices in mathematics or spreadsheet data.

### Creating 2D Lists

**Figure 7.1: Creating 2D Lists**

In [ ]:
# 2D list (matrix)
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

print("Matrix:")
for row in matrix:
    print(row)

# Dimensions
rows = len(matrix)
cols = len(matrix[0])
print(f"\nDimensions: {rows} x {cols}")

# Access individual elements
print(f"\nmatrix[0][0] = {matrix[0][0]}")  # Row 0, Col 0
print(f"matrix[1][2] = {matrix[1][2]}")  # Row 1, Col 2
print(f"matrix[2][1] = {matrix[2][1]}")  # Row 2, Col 1

### Iterating 2D Lists

**Figure 7.2: Iterating 2D Lists**

In [ ]:
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

# Method 1: Nested loops
print("All elements:")
for row in matrix:
    for element in row:
        print(element, end=" ")
    print()  # New line after each row

# Method 2: With indices
print("\nWith indices:")
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        print(f"[{i}][{j}]={matrix[i][j]}", end="  ")
    print()

# Sum all elements
total = sum(sum(row) for row in matrix)
print(f"\nSum of all elements: {total}")

### Matrix Operations

**Figure 7.3: Matrix Operations**

In [ ]:
# Row and column sums
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

# Row sums
print("Row sums:")
for i, row in enumerate(matrix):
    print(f"  Row {i}: {sum(row)}")

# Column sums
print("\nColumn sums:")
for j in range(len(matrix[0])):
    col_sum = sum(matrix[i][j] for i in range(len(matrix)))
    print(f"  Col {j}: {col_sum}")

# Diagonal sum (main diagonal)
diag_sum = sum(matrix[i][i] for i in range(len(matrix)))
print(f"\nMain diagonal sum: {diag_sum}")

# Transpose
transposed = [[matrix[j][i] for j in range(len(matrix))] 
              for i in range(len(matrix[0]))]
print("\nTransposed:")
for row in transposed:
    print(row)

#### Creating Empty 2D Lists

**Figure 7.4: Creating Empty 2D Lists**

In [ ]:
# CORRECT way to create 2D list
rows, cols = 3, 4
matrix = [[0 for _ in range(cols)] for _ in range(rows)]
print("Empty 3x4 matrix:")
for row in matrix:
    print(row)

# Modify one element
matrix[1][2] = 99
print("\nAfter matrix[1][2] = 99:")
for row in matrix:
    print(row)

# WRONG way (creates references to same list!)
# wrong = [[0] * cols] * rows  # Don't do this!
# Changing one row would change all rows

---
## Part 8: Introduction to Tuples


A **tuple** is an ordered, **immutable** sequence of elements. Once created, a tuple cannot be modified. Tuples use parentheses `()` instead of square brackets.

### Tuples vs Lists

| Feature | List | Tuple |
| --- | --- | --- |
| **Syntax** | `[1, 2, 3]` | `(1, 2, 3)` |
| **Mutability** | Mutable (can change) | Immutable (cannot change) |
| **Use case** | Data that may change | Data that shouldn't change |
| **Performance** | Slightly slower | Slightly faster |
| **Methods** | Many (append, etc.) | Few (count, index) |

### Creating Tuples

**Figure 8.1: Creating Tuples**

In [ ]:
# Creating tuples
empty = ()
single = (42,)  # Note the comma!
coordinates = (10, 20)
rgb = (255, 128, 0)
mixed = (1, "hello", 3.14, True)

print("Empty tuple:", empty)
print("Single element:", single, "- type:", type(single))
print("Coordinates:", coordinates)
print("RGB:", rgb)
print("Mixed:", mixed)

# Without parentheses (tuple packing)
point = 5, 10, 15
print("\nPacked tuple:", point, "- type:", type(point))

# From list
from_list = tuple([1, 2, 3])
print("From list:", from_list)

> 💡 **Note:** **Single-element tuple:** Must include a trailing comma: `(42,)` is a tuple, but `(42)` is just the integer 42 in parentheses.

### Accessing Tuple Elements

**Figure 8.2: Tuple Indexing and Slicing**

In [ ]:
# Indexing works just like lists
colors = ("red", "green", "blue", "yellow")

print("Index 0:", colors[0])
print("Index -1:", colors[-1])
print("Slice [1:3]:", colors[1:3])

# Length, min, max work too
numbers = (5, 2, 8, 1, 9)
print(f"\nTuple: {numbers}")
print(f"Length: {len(numbers)}")
print(f"Min: {min(numbers)}")
print(f"Max: {max(numbers)}")
print(f"Sum: {sum(numbers)}")

### Tuple Immutability

**Figure 8.3: Tuples are Immutable**

In [ ]:
# Tuples cannot be modified
point = (10, 20, 30)
print("Original:", point)

# This would cause an error:
# point[0] = 99  # TypeError!

# But you can create a new tuple
point = point + (40,)
print("New tuple with 40:", point)

# Or convert to list, modify, convert back
coords = (1, 2, 3)
temp_list = list(coords)
temp_list[1] = 99
coords = tuple(temp_list)
print("Modified via list:", coords)

# Check if element exists
print(f"\n20 in point: {20 in point}")
print(f"Index of 20: {point.index(20)}")

---
## Part 9: Tuple Operations and Unpacking


### Tuple Unpacking

One of the most useful features of tuples is **unpacking** - assigning multiple variables at once.

**Figure 9.1: Tuple Unpacking**

In [ ]:
# Basic unpacking
coordinates = (10, 20, 30)
x, y, z = coordinates
print(f"x={x}, y={y}, z={z}")

# Swap variables (Python magic!)
a, b = 5, 10
print(f"\nBefore swap: a={a}, b={b}")
a, b = b, a
print(f"After swap: a={a}, b={b}")

# Unpacking in for loops
points = [(1, 2), (3, 4), (5, 6)]
print("\nPoints:")
for x, y in points:
    print(f"  x={x}, y={y}")

# Unpacking function returns
def get_stats(numbers):
    return min(numbers), max(numbers), sum(numbers)/len(numbers)

data = [23, 45, 67, 89, 12]
minimum, maximum, average = get_stats(data)
print(f"\nMin: {minimum}, Max: {maximum}, Avg: {average:.1f}")

### Extended Unpacking with *

**Figure 9.2: Extended Unpacking**

In [ ]:
# Use * to capture remaining elements
numbers = (1, 2, 3, 4, 5, 6, 7)

first, *middle, last = numbers
print(f"First: {first}")
print(f"Middle: {middle}")
print(f"Last: {last}")

# Get first and rest
head, *tail = numbers
print(f"\nHead: {head}")
print(f"Tail: {tail}")

# Get all except last
*init, last = numbers
print(f"\nInit: {init}")
print(f"Last: {last}")

### Named Tuples

**Figure 9.3: Named Tuples**

In [ ]:
from collections import namedtuple

# Create a named tuple class
Point = namedtuple('Point', ['x', 'y', 'z'])
Color = namedtuple('Color', ['red', 'green', 'blue'])

# Create instances
p1 = Point(10, 20, 30)
orange = Color(255, 165, 0)

# Access by name or index
print(f"Point: x={p1.x}, y={p1.y}, z={p1.z}")
print(f"Point: p1[0]={p1[0]}")

print(f"\nOrange: R={orange.red}, G={orange.green}, B={orange.blue}")

# Still immutable!
# p1.x = 99  # AttributeError!

# Useful for readable code
Student = namedtuple('Student', ['name', 'id', 'gpa'])
alice = Student("Alice", "12345", 3.8)
print(f"\n{alice.name} (ID: {alice.id}): GPA = {alice.gpa}")

### Tuple Methods

**Figure 9.4: Tuple Methods**

In [ ]:
# Tuples have only 2 methods
grades = (85, 90, 78, 85, 92, 85, 88)

# count() - count occurrences
count_85 = grades.count(85)
print(f"Number of 85s: {count_85}")

# index() - find first occurrence
first_85 = grades.index(85)
print(f"First 85 at index: {first_85}")

# Membership testing
print(f"\n90 in grades: {90 in grades}")
print(f"100 in grades: {100 in grades}")

# Concatenation creates new tuple
t1 = (1, 2, 3)
t2 = (4, 5, 6)
combined = t1 + t2
print(f"\nCombined: {combined}")

---
## Part 10: Lists vs Tuples: When to Use Each


### Use Lists When:

- Data needs to be modified (add, remove, change)
- Order matters but may change
- Working with homogeneous data (all same type)
- Building collections dynamically

### Use Tuples When:

- Data should not change (coordinates, constants)
- Returning multiple values from functions
- Using as dictionary keys (lists can't be keys!)
- Representing fixed structures (RGB, coordinates)
- Slight performance optimization needed

**Figure 10.1: Practical Examples**

In [ ]:
# LIST: shopping cart (items change)
cart = ["apple", "bread"]
cart.append("milk")
cart.remove("bread")
print("Shopping cart:", cart)

# TUPLE: coordinates (shouldn't change)
origin = (0, 0, 0)
destination = (10, 20, 5)
print(f"From {origin} to {destination}")

# TUPLE: function returning multiple values
def divide_with_remainder(a, b):
    return a // b, a % b

quotient, remainder = divide_with_remainder(17, 5)
print(f"\n17 ÷ 5 = {quotient} remainder {remainder}")

# TUPLE: dictionary key (lists can't be keys!)
locations = {
    (0, 0): "Origin",
    (10, 20): "Office",
    (5, 5): "Cafe"
}
print(f"\nLocation at (10, 20): {locations[(10, 20)]}")

**Figure 10.2: Performance Comparison**

In [ ]:
import sys

# Memory comparison
my_list = [1, 2, 3, 4, 5]
my_tuple = (1, 2, 3, 4, 5)

print(f"List size: {sys.getsizeof(my_list)} bytes")
print(f"Tuple size: {sys.getsizeof(my_tuple)} bytes")
print(f"Tuple is {sys.getsizeof(my_list) - sys.getsizeof(my_tuple)} bytes smaller")

# Tuples are hashable, lists are not
print(f"\nHash of tuple: {hash(my_tuple)}")
# print(hash(my_list))  # TypeError: unhashable type: 'list'

# This means tuples can be used in sets
point_set = {(0, 0), (1, 1), (2, 2), (0, 0)}  # Duplicates removed
print(f"Unique points: {point_set}")

---
## Part 11: Engineering Applications


#### Sensor Data Processing

**Figure 11.1: Sensor Data**

In [ ]:
# Processing temperature sensor readings
readings = [23.5, 24.1, 23.8, 24.2, 23.9, 24.0, 23.7, 24.3]

def analyze_sensor_data(data):
    """Analyze sensor readings and return statistics"""
    n = len(data)
    avg = sum(data) / n
    variance = sum((x - avg) ** 2 for x in data) / n
    std_dev = variance ** 0.5
    
    return {
        'count': n,
        'min': min(data),
        'max': max(data),
        'average': round(avg, 2),
        'std_dev': round(std_dev, 2),
        'range': round(max(data) - min(data), 2)
    }

stats = analyze_sensor_data(readings)
print("📊 Sensor Analysis:")
for key, value in stats.items():
    print(f"  {key}: {value}")

#### 3D Coordinates and Vectors

**Figure 11.2: Vector Operations**

In [ ]:
import math

def vector_add(v1, v2):
    """Add two vectors"""
    return tuple(a + b for a, b in zip(v1, v2))

def vector_magnitude(v):
    """Calculate vector magnitude"""
    return math.sqrt(sum(x**2 for x in v))

def vector_dot(v1, v2):
    """Calculate dot product"""
    return sum(a * b for a, b in zip(v1, v2))

# Example vectors
v1 = (3, 4, 0)
v2 = (1, 2, 2)

print(f"Vector 1: {v1}")
print(f"Vector 2: {v2}")
print(f"Sum: {vector_add(v1, v2)}")
print(f"|v1|: {vector_magnitude(v1):.2f}")
print(f"|v2|: {vector_magnitude(v2):.2f}")
print(f"v1 · v2: {vector_dot(v1, v2)}")

#### Matrix Multiplication

**Figure 11.3: Matrix Multiplication**

In [ ]:
def matrix_multiply(A, B):
    """Multiply two matrices"""
    rows_A, cols_A = len(A), len(A[0])
    rows_B, cols_B = len(B), len(B[0])
    
    if cols_A != rows_B:
        return None
    
    # Create result matrix
    result = [[0 for _ in range(cols_B)] for _ in range(rows_A)]
    
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]
    
    return result

# Example: 2x3 × 3x2 = 2x2
A = [[1, 2, 3],
     [4, 5, 6]]

B = [[7, 8],
     [9, 10],
     [11, 12]]

print("Matrix A:")
for row in A: print(row)

print("\nMatrix B:")
for row in B: print(row)

C = matrix_multiply(A, B)
print("\nA × B =")
for row in C: print(row)

---
## ❌ Common Mistakes to Avoid


These are the most frequent errors students make with lists and tuples. Study them before the exercises!

**Index out of range**

                    A list with 5 elements has indices 0–4. Accessing `my_list[5]` raises `IndexError`. Remember: the last valid index is `len(my_list) - 1`, or use `my_list[-1]`.

**`append()` vs `extend()`**

`lst.append([4,5])` adds the list as a single element: `[1, 2, 3, [4, 5]]`.

`lst.extend([4,5])` adds each element individually: `[1, 2, 3, 4, 5]`.

**Shallow copy trap**

`new_list = old_list` does NOT create a copy — both names point to the same list! Use `new_list = old_list[:]` or `new_list = old_list.copy()` for a true copy.

**`sort()` returns `None`**

`result = my_list.sort()` → `result` is `None`! `.sort()` modifies the list in-place. If you need a sorted copy, use `result = sorted(my_list)`.

**Trying to modify a tuple**

`my_tuple[0] = 99` → `TypeError`! Tuples are immutable. If you need to change values, convert to a list first or create a new tuple.

---
# 📝 Exercises


### Exercise 1: Create a List  (Easy)

Create a list called `numbers` containing 1 through 5, then print it.

**Expected Output:**
```
[1, 2, 3, 4, 5]
```

<details>
<summary>💡 Hints</summary>

- Use square brackets: `[1, 2, 3, 4, 5]`
- Or use: `list(range(1, 6))`
- Assign to variable: `numbers = [...]`
</details>

In [ ]:
# ✏️ [EX1]
# Create the list

print(numbers)

### Exercise 2: Access Last Element  (Easy)

Print the last element of the list using negative indexing.

**Expected Output:**
```
elderberry
```

<details>
<summary>💡 Hints</summary>

- Negative index -1 gets last element
- Syntax: `fruits[-1]`
- -2 would get second-to-last, etc.
</details>

In [ ]:
# ✏️ [EX2]
fruits = ["apple", "banana", "cherry", "date", "elderberry"]
# Print the last element using negative indexing

### Exercise 3: Slice Middle Elements  (Easy)

Extract elements at indices 2, 3, 4 (the middle three numbers).

**Expected Output:**
```
[30, 40, 50]
```

<details>
<summary>💡 Hints</summary>

- Slice syntax: `list[start:end]`
- End index is exclusive, so use `[2:5]`
- Gets indices 2, 3, 4 (values 30, 40, 50)
</details>

In [ ]:
# ✏️ [EX3]
numbers = [10, 20, 30, 40, 50, 60, 70]
# Slice to get [30, 40, 50]
middle = 
print(middle)

### Exercise 4: Append and Extend  (Easy)

Add "date" using append, then add ["elderberry", "fig"] using extend.

**Expected Output:**
```
['apple', 'banana', 'cherry', 'date', 'elderberry', 'fig']
```

<details>
<summary>💡 Hints</summary>

- Append single item: `fruits.append("date")`
- Extend with list: `fruits.extend(["elderberry", "fig"])`
- append() adds one, extend() adds multiple
</details>

In [ ]:
# ✏️ [EX4]
fruits = ["apple", "banana", "cherry"]
# Add "date"

# Add ["elderberry", "fig"]

print(fruits)

### Exercise 5: List Sum and Average  (Medium)

Calculate and print the sum and average of the list.

**Expected Output:**
```
Sum: 45
Average: 9.0
```

<details>
<summary>💡 Hints</summary>

- Use `sum(numbers)` for total
- Average: `sum(numbers) / len(numbers)`
- Assign to variables: total and avg
</details>

In [ ]:
# ✏️ [EX5]
numbers = [5, 10, 15, 7, 8]
# Calculate sum and average

print(f"Sum: {total}")
print(f"Average: {avg}")

### Exercise 6: Reverse a List  (Easy)

Reverse the list using slicing (not the reverse() method).

**Expected Output:**
```
[5, 4, 3, 2, 1]
```

<details>
<summary>💡 Hints</summary>

- Use slice with step -1: `[::-1]`
- Syntax: `numbers[::-1]`
- Creates a reversed copy of the list
</details>

In [ ]:
# ✏️ [EX6]
numbers = [1, 2, 3, 4, 5]
# Reverse using slicing
reversed_list = 
print(reversed_list)

### Exercise 7: List Comprehension Squares  (Medium)

Use a list comprehension to create squares of 1-5.

**Expected Output:**
```
[1, 4, 9, 16, 25]
```

<details>
<summary>💡 Hints</summary>

- Comprehension: `[expression for item in iterable]`
- Square: `x ** 2` or `x * x`
- Example: `[x**2 for x in range(1, 6)]`
</details>

In [ ]:
# ✏️ [EX7]
# Create squares using comprehension
squares = 
print(squares)

### Exercise 8: Filter Even Numbers  (Medium)

Use a list comprehension to filter only even numbers.

**Expected Output:**
```
[2, 4, 6, 8, 10]
```

<details>
<summary>💡 Hints</summary>

- Add condition: `[x for x in list if condition]`
- Even check: `x % 2 == 0`
- Example: `[x for x in numbers if x % 2 == 0]`
</details>

In [ ]:
# ✏️ [EX8]
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# Filter even numbers using comprehension
evens = 
print(evens)

### Exercise 9: 2D List Row Sum  (Medium)

Calculate and print the sum of each row in the matrix.

**Expected Output:**
```
Row 0 sum: 6
Row 1 sum: 15
Row 2 sum: 24
```

<details>
<summary>💡 Hints</summary>

- Use `for i, row in enumerate(matrix):`
- Sum each row: `sum(row)`
- Print: `f"Row {i} sum: {sum(row)}"`
</details>

In [ ]:
# ✏️ [EX9]
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]
# Print sum of each row

### Exercise 10: Tuple Unpacking  (Easy)

Unpack the coordinates tuple into x, y, z variables.

**Expected Output:**
```
x=10, y=20, z=30
```

<details>
<summary>💡 Hints</summary>

- Unpack syntax: `x, y, z = tuple`
- One line: `x, y, z = coordinates`
- Number of variables must match tuple length
</details>

In [ ]:
# ✏️ [EX10]
coordinates = (10, 20, 30)
# Unpack into x, y, z

print(f"x={x}, y={y}, z={z}")

### Exercise 11: Swap Variables  (Easy)

Swap the values of a and b using tuple unpacking.

**Expected Output:**
```
Before: a=5, b=10
After: a=10, b=5
```

<details>
<summary>💡 Hints</summary>

- Python swap idiom: `a, b = b, a`
- No temp variable needed
- Works with tuples on both sides
</details>

In [ ]:
# ✏️ [EX11]
a = 5
b = 10
print(f"Before: a={a}, b={b}")

# Swap a and b

print(f"After: a={a}, b={b}")

### Exercise 12: Sort and Find Max  (Medium)

Sort the list and print the top 3 scores.

**Expected Output:**
```
Top 3 scores: [95, 92, 88]
```

<details>
<summary>💡 Hints</summary>

- Sort descending: `sorted(scores, reverse=True)`
- Take first 3: `[:3]`
- Combine: `sorted(scores, reverse=True)[:3]`
</details>

In [ ]:
# ✏️ [EX12]
scores = [78, 92, 65, 88, 72, 95, 81]
# Get top 3 scores (highest first)
top3 = 
print(f"Top 3 scores: {top3}")

### Exercise 13: Enumerate Loop  (Medium)

Use enumerate to print each fruit with its position (starting from 1).

**Expected Output:**
```
1. apple
2. banana
3. cherry
```

<details>
<summary>💡 Hints</summary>

- Use `enumerate(fruits, start=1)`
- Unpack: `for i, fruit in enumerate(...):`
- Print: `f"{i}. {fruit}"`
</details>

In [ ]:
# ✏️ [EX13]
fruits = ["apple", "banana", "cherry"]
# Use enumerate starting from 1

### Exercise 14: Flatten 2D List  (Challenge)

Flatten a 2D list into a 1D list using list comprehension.

**Expected Output:**
```
[1, 2, 3, 4, 5, 6, 7, 8, 9]
```

<details>
<summary>💡 Hints</summary>

- Use nested comprehension: `[item for sublist in matrix for item in sublist]`
</details>

In [ ]:
# ✏️ [EX14]
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
# Flatten using comprehension
flat = 
print(flat)

### Exercise 15: Matrix Transpose  (Challenge)

Transpose the matrix (swap rows and columns).

**Expected Output:**
```
[[1, 4], [2, 5], [3, 6]]
```

<details>
<summary>💡 Hints</summary>

- Use zip() with * unpacking: `list(map(list, zip(*matrix)))`
- Or nested comprehension
</details>

In [ ]:
# ✏️ [EX15]
matrix = [[1, 2, 3], [4, 5, 6]]
# Transpose the matrix
transposed = 
print(transposed)

### Exercise 🌉: Bridge Exercise: Sneak Peek at Week 6  (Preview)

**Next week: Dictionaries!** You have two parallel lists — sensor names and their readings. Keeping them in sync is fragile and error-prone. What if each name could be directly linked to its reading?

**Expected Output:**
```
Sensor: temp_01 → Reading: 24.5
Sensor: humidity_01 → Reading: 65.2
Sensor: pressure_01 → Reading: 1013.25

Problem: What if we delete from one list but not the other?
Names: ['temp_01', 'humidity_01']
Readings: [24.5, 65.2, 1013.25]
Data is now mismatched! 😫
Next week: dictionaries link keys to values — no sync issues!
```

<details>
<summary>💡 Hints</summary>

- Parallel lists require you to use the same index for both — easy to break
- Deleting from one list without the other causes data mismatch
- Next week: `{"temp_01": 24.5, "humidity_01": 65.2}` — each key directly maps to its value!
</details>

In [ ]:
# ✏️ [EXBridge]
# Bridge Exercise: Parallel Lists Problem
# Keeping two related lists in sync is fragile!

sensor_names = ["temp_01", "humidity_01", "pressure_01"]
sensor_readings = [24.5, 65.2, 1013.25]

# Display paired data using index
for i in range(len(sensor_names)):
    print(f"Sensor: {sensor_names[i]} → Reading: {sensor_readings[i]}")

# Now delete from ONE list...
sensor_names.remove("pressure_01")
# Oops! We forgot to remove from the other list!

print("\nProblem: What if we delete from one list but not the other?")
print(f"Names: {sensor_names}")
print(f"Readings: {sensor_readings}")
print("Data is now mismatched! 😫")
print("Next week: dictionaries link keys to values — no sync issues!")

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {
    "ex1": "8f7b1e39380505a8eb29703b08643fa2",
    "ex2": "1346f853155bc4d1f588b9d90b69a18a",
    "ex3": "7f888f341fd195eecca9cd3b33a1c5c0",
    "ex4": "def04da9049883e206ab54bfce091bad",
    "ex5": "419006ba092e9f65b16cd70e0b6deb7a",
    "ex6": "0304e5b7e406e2f7aae2a5a41aa52ed3",
    "ex7": "65b1681c724c8fe9ee8e62e8580e810b",
    "ex8": "5d56996cb21563516befad22e87670f9",
    "ex9": "bf7ef9c56d30a4674a4410bcd63d75d4",
    "ex10": "edf98109031dfb95f250705a422a95f7",
    "ex11": "cf5a53880bab3df8c029400bc933bbf4",
    "ex12": "5436f1bc32ba829beb9239159a86a380",
    "ex13": "5e72014753a99fcc49154dfe4375d821",
    "ex14": "28e64fa0892aa846e5a63574874dbb89",
    "ex15": "428f97bd4baa95af1f5c0155e77a134c"
}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request, hashlib
import time as _time, datetime as _dt

WEEK = "Week_05"
URL  = "https://script.google.com/macros/s/AKfycbyf1D3HGSAX4MoIhNlAuWlGrFyyvbM5MIv7ZsLxrVDlATUihrRGEAaibvIZYlCfd8Me/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'cp2-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
